In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import activations
from tensorflow.keras import layers
import numpy as np 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 

# This Jupyter Notebook is the projection of my thought process
## I will be telling you where all the informations are from 
### (for example : "according to the paper, ...") 


# Let's use the CIFAR Dataset


In [70]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

LABELS = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
HEIGHT = x_train.shape[1]
WIDTH = x_train.shape[2]
INPUT_SHAPE = (HEIGHT, WIDTH, 3)

(50000, 32, 32, 3)
(50000, 1)


# The actual Paper ! 

# Let's try to code one residual block

### Based on the paper : f(x) + x is the output of our residual unit + input ( f(x) being the residual unit)

###### Let's go for Conv -> Bn -> Relu -> Conv -> Bn -> Addition -> Relu :)

In [71]:
def residual_block(inputs, kernel_size, filters, reduce=False): 
    x = inputs
    stride = 1
    print("Input shapes : ",x.shape)
    if reduce: 
        stride = 2 
        #let's reduce dimensiosn 
        print("reducing ...")
        x = tf.keras.layers.Conv2D(
            filters=filters[0], 
            kernel_size=1, 
            padding="SAME", 
            strides=2
        )(x)
    #x = x if not reduce else  x = x with stride /2 
    #conv operations ! 
    result = inputs
    for step in range(len(kernel_size)-1): 
        print("step : ",step)
        if step == 0 and reduce: 
            print("stride of 2")
            stride = 2 
        else: 
            stride = 1
            print("stride of 1")
        result = tf.keras.layers.Conv2D(
            filters=filters[step], 
            kernel_size=kernel_size[step], 
            padding="SAME", 
            strides=stride
        )(result)
        result = tf.keras.layers.BatchNormalization()(result)
    #for the last conv layer : Conv -> bn -> addition
    result = tf.keras.layers.Conv2D(
            filters=filters[len(kernel_size)-1], 
            kernel_size=kernel_size[len(kernel_size)-1], 
            padding="SAME", 
            strides=1
        )(result)
    result = tf.keras.layers.BatchNormalization()(result)
    #Addition 
    print(result.shape)
    print(x.shape)
    result = result + x
    reslut = tf.keras.activations.relu(x)
    print(result.shape)
    print("done")
    return result 

## Let's try our residual_block with the original paper's dimensions ! We expect our output to be 28 * 28 * 128 (input being 56,56,64)

In [72]:
test_input_shape = (56,56,64)
test_input = tf.keras.Input(shape=test_input_shape)
kernel_size = [3,3]
filters = [128,128]

res = test_input
for i in range(4): 
    print("block ", i+1)
    if i == 0: 
        res = residual_block(res, kernel_size, filters, reduce=True)
    else: 
        res = residual_block(res, kernel_size, filters, reduce=False)
print(res.shape)

block  1
Input shapes :  (None, 56, 56, 64)
reducing ...
step :  0
stride of 2
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
block  2
Input shapes :  (None, 28, 28, 128)
step :  0
stride of 1
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
block  3
Input shapes :  (None, 28, 28, 128)
step :  0
stride of 1
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
block  4
Input shapes :  (None, 28, 28, 128)
step :  0
stride of 1
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
(None, 28, 28, 128)


# Look ! It's exactly working as we wanted ! 
## But it's not the end ! We have done the hardest !
## Let's implement the 34 layers architectures from the paper

<img src="dimensions_test.PNG"></img>

# The model is saying : input = 224 x 224 x 3 ! 

In [120]:
def resnet_34_layers(inputs): 
        
    x = inputs 
    #conv1 + Max pool
    x = tf.keras.layers.Conv2D(
        filters=64, 
        kernel_size=7, 
        strides=2, 
        padding="SAME", 
        input_shape=inputs.shape
    )(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding="SAME")(x)
    x.shape
    
    #Let's use our residual_blocks ! 
    
    x = residual_block(x, kernel_size=[3,3], filters=[64,64], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[64,64], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[64,64], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[128,128], reduce=True)
    x = residual_block(x, kernel_size=[3,3], filters=[128,128], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[128,128], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[128,128], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[256,256], reduce=True)
    x = residual_block(x, kernel_size=[3,3], filters=[256,256], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[256,256], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[256,256], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[256,256], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[256,256], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[512,512], reduce=True)
    x = residual_block(x, kernel_size=[3,3], filters=[512,512], reduce=False)
    x = residual_block(x, kernel_size=[3,3], filters=[512,512], reduce=False)
    
    #Average Pooling ! 
    x = tf.keras.layers.MaxPooling2D()(x)
    #fc 
    x = tf.keras.layers.Flatten()(x)
    outputs  = tf.keras.layers.Dense(units=1000, activation="sigmoid")(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [121]:
inputs = tf.keras.Input(shape=(224,224,3))
print(inputs)
model = resnet_34_layers(inputs)

Tensor("input_54:0", shape=(None, 224, 224, 3), dtype=float32)
Input shapes :  (None, 56, 56, 64)
step :  0
stride of 1
(None, 56, 56, 64)
(None, 56, 56, 64)
(None, 56, 56, 64)
done
Input shapes :  (None, 56, 56, 64)
step :  0
stride of 1
(None, 56, 56, 64)
(None, 56, 56, 64)
(None, 56, 56, 64)
done
Input shapes :  (None, 56, 56, 64)
step :  0
stride of 1
(None, 56, 56, 64)
(None, 56, 56, 64)
(None, 56, 56, 64)
done
Input shapes :  (None, 56, 56, 64)
reducing ...
step :  0
stride of 2
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
Input shapes :  (None, 28, 28, 128)
step :  0
stride of 1
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
Input shapes :  (None, 28, 28, 128)
step :  0
stride of 1
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
Input shapes :  (None, 28, 28, 128)
step :  0
stride of 1
(None, 28, 28, 128)
(None, 28, 28, 128)
(None, 28, 28, 128)
done
Input shapes :  (None, 28, 28, 128)
reducing ...
step :  0
stride of 2
(None

In [122]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_54 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_620 (Conv2D)             (None, 112, 112, 64) 9472        input_54[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_25 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_620[0][0]                 
__________________________________________________________________________________________________
conv2d_621 (Conv2D)             (None, 56, 56, 64)   36928       max_pooling2d_25[0][0]           
___________________________________________________________________________________________

# We have the exact same summary as in the paper  ! 
